<a href="https://colab.research.google.com/github/joeyMckinney/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/josiah_mckinney_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# Importing data

In [508]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [509]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [510]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [511]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

#Wrangle data

In [512]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [513]:
df.shape

(23040, 21)

In [514]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNITS                

In [515]:
#get only rows where building class category is 01 one family dwellings
condition = df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'
condition2 = (df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)
df2 = df[condition & condition2]

In [516]:
df2.shape

(3164, 21)

In [517]:
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [518]:
df2.isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING_CLASS_CATEGORY              0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
EASE-MENT                         3164
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  3163
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

In [519]:
df2['TAX_CLASS_AT_TIME_OF_SALE'].value_counts()

1    3164
Name: TAX_CLASS_AT_TIME_OF_SALE, dtype: int64

In [520]:
df2 = df2.drop(['EASE-MENT', 'APARTMENT_NUMBER', 'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_CATEGORY','ADDRESS'], axis=1)

In [521]:
df2['LAND_SQUARE_FEET'] = df2['LAND_SQUARE_FEET'].str.replace(',', '')

In [522]:
df2['LAND_SQUARE_FEET'] = df2['LAND_SQUARE_FEET'].astype(int)

In [523]:
df2.shape

(3164, 16)

#Spliting data

In [524]:
#making sale date a datatime format
df2['SALE_DATE'] = pd.to_datetime(df2['SALE_DATE'], infer_datetime_format=True)

In [525]:
df2['SALE_DATE']

44      2019-01-01
61      2019-01-01
78      2019-01-02
108     2019-01-02
111     2019-01-02
           ...    
23029   2019-04-30
23031   2019-04-30
23032   2019-04-30
23033   2019-04-30
23035   2019-04-30
Name: SALE_DATE, Length: 3164, dtype: datetime64[ns]

In [526]:
#making my traing set and validation set
cutoff = '2019-04-01'
mask = df2.SALE_DATE < cutoff

X = df2.drop(['SALE_PRICE'], axis=1)
y = df2[['SALE_PRICE', 'SALE_DATE']]

X_train, y_train = X.loc[mask], y.loc[mask]
X_val, y_val = X.loc[~mask], y.loc[~mask] 

In [527]:
X.shape

(3164, 15)

In [528]:
y.shape

(3164, 2)

In [529]:
#checking work
assert X_train.shape[0] + X_val.shape[0] == X.shape[0]

In [530]:
X_train.shape

(2517, 15)

In [531]:
X_val.shape

(647, 15)

#Establish  Baseline

In [532]:
from sklearn.metrics import mean_absolute_error

In [533]:
baseline = mean_absolute_error(y_train['SALE_PRICE'], [y_train['SALE_PRICE'].mean()]*len(y_train['SALE_PRICE']))
baseline

216497.12357684076

#Build Model

In [534]:
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression

In [535]:
X_train.head()

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
44,3,OTHER,1,5495,801,A9,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,A9,2019-01-01
61,4,OTHER,1,7918,72,A1,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,A1,2019-01-01
78,2,OTHER,1,4210,19,A1,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,A1,2019-01-02
108,3,OTHER,1,5212,69,A1,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,A1,2019-01-02
111,3,OTHER,1,7930,121,A5,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,A5,2019-01-02


In [536]:
X_train.dtypes

BOROUGH                                   object
NEIGHBORHOOD                              object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
BUILDING_CLASS_AT_PRESENT                 object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                           int64
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_DATE                         datetime64[ns]
dtype: object

##One Hot Encoder

In [537]:
ohe = OneHotEncoder(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE'], use_cat_names=True)

XT_train = ohe.fit_transform(X_train)
XT_val = ohe.transform(X_val)

In [538]:
XT_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0,SALE_DATE
44,1,0,0,0,0,1,0,0,0,0,0,0,1,0,5495,801,1,0,0,0,0,0,0,0,0,0,0,0,0,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,1,0,0,0,0,0,0,0,0,0,0,2019-01-01
61,0,1,0,0,0,1,0,0,0,0,0,0,1,0,7918,72,0,1,0,0,0,0,0,0,0,0,0,0,0,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,0,1,0,0,0,0,0,0,0,0,0,2019-01-01
78,0,0,1,0,0,1,0,0,0,0,0,0,1,0,4210,19,0,1,0,0,0,0,0,0,0,0,0,0,0,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,0,1,0,0,0,0,0,0,0,0,0,2019-01-02
108,1,0,0,0,0,1,0,0,0,0,0,0,1,0,5212,69,0,1,0,0,0,0,0,0,0,0,0,0,0,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,0,1,0,0,0,0,0,0,0,0,0,2019-01-02
111,1,0,0,0,0,1,0,0,0,0,0,0,1,0,7930,121,0,0,1,0,0,0,0,0,0,0,0,0,0,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,0,0,1,0,0,0,0,0,0,0,0,2019-01-02


In [539]:
XT_train.shape

(2517, 48)

##SelectKBest

In [540]:
#have to remove date time object to use selectKBest
XT_train = XT_train.drop(['SALE_DATE'], axis=1)

In [541]:
skb = SelectKBest(k=5)

XTT_train = skb.fit_transform(XT_train, y_train['SALE_PRICE'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


## Ridge Regression

In [542]:
from sklearn.linear_model import Ridge

model = Ridge(normalize=True)

model.fit(XT_train, y_train['SALE_PRICE'])

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [543]:
#droping data time objects
XT_val = XT_val.drop(['SALE_DATE'], axis=1)

In [544]:
#droping data time objects
y_val = y_val.drop(['SALE_DATE'], axis=1)

In [545]:
#getting R2 score
model.score(XT_val, y_val)

0.31924715727630304

#getting MAE

In [561]:
ks = list(range(1,len(X_train.columns)+1))
r2_scores = []

for k in ks:

  ohe2 = OneHotEncoder(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_TIME_OF_SALE'], use_cat_names=True)

  XT_train2 = ohe2.fit_transform(X_train)

  skb2 = SelectKBest(k=k)

  XT_train2 = XT_train2.drop(['SALE_DATE'], axis=1)

  XTT_train2 = skb2.fit_transform(XT_train2, y_train['SALE_PRICE'])

  model2 = LinearRegression()

  model2.fit(XTT_train2, y_train['SALE_PRICE'])


  r2_scores.append(model2.score(XTT_train2, y_train['SALE_PRICE']))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  

In [562]:
ks

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [563]:
r2_scores

[0.007764827932696949,
 0.00826970521828041,
 0.008243412844198827,
 0.007544874063768248,
 0.01567943388792903,
 0.07508897079268528,
 0.07393396480472936,
 0.0888885800835193,
 0.09899122494494872,
 0.097643623393733,
 0.24507876779772975,
 0.2547347631075356,
 0.26138202406736055,
 0.26171657915774493,
 0.26250786860378883]

In [567]:
import matplotlib.pyplot as plt
plt.plot(ks, r2_scores)
plt.xlabel('k (number of features selected)')
plt.ylabel('$R^2$ score');
plt.show()

In [560]:
from sklearn.metrics import mean_absolute_error

In [568]:
skb = SelectKBest(k=13)

XTT_train = skb.fit_transform(XT_train, y_train['SALE_PRICE'])

model2 = LinearRegression()

model2.fit(XTT_train, y_train['SALE_PRICE'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [569]:
y_predict = model2.predict(XTT_train)

In [573]:
model2_mae = mean_absolute_error(y_train['SALE_PRICE'], y_predict)
model2_mae

186728.6976171867

In [574]:
baseline

216497.12357684076